In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [3]:
import time
import os

In [4]:
train_transforms=transforms.Compose([transforms.ToTensor(), # Need to run Part 1 of the image transforms before this
                                     transforms.Normalize([0.485, 0.456, 0.406], 
                                                          [0.229, 0.224, 0.225])])
test_transforms=transforms.Compose([transforms.Resize(224), 
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])

In [5]:
path = '..\\Data\\CATS_DOGS\\'
train_data=datasets.ImageFolder(root=path+'train1',transform=train_transforms)
test_data=datasets.ImageFolder(root=path+'test',transform=test_transforms)

labels=train_data.classes
print(labels)

['CAT', 'DOG']


In [6]:
train_loader=DataLoader(train_data,shuffle=True, batch_size=500, num_workers=4)
test_loader=DataLoader(test_data,shuffle=False, batch_size=10, num_workers=4)

In [7]:
resnet50_model=models.resnet50(pretrained=True)

In [8]:
resnet50_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
for params in resnet50_model.parameters():
    print(params.numel())
    params.requires_grad=False

9408
64
64
4096
64
64
36864
64
64
16384
256
256
16384
256
256
16384
64
64
36864
64
64
16384
256
256
16384
64
64
36864
64
64
16384
256
256
32768
128
128
147456
128
128
65536
512
512
131072
512
512
65536
128
128
147456
128
128
65536
512
512
65536
128
128
147456
128
128
65536
512
512
65536
128
128
147456
128
128
65536
512
512
131072
256
256
589824
256
256
262144
1024
1024
524288
1024
1024
262144
256
256
589824
256
256
262144
1024
1024
262144
256
256
589824
256
256
262144
1024
1024
262144
256
256
589824
256
256
262144
1024
1024
262144
256
256
589824
256
256
262144
1024
1024
262144
256
256
589824
256
256
262144
1024
1024
524288
512
512
2359296
512
512
1048576
2048
2048
2097152
2048
2048
1048576
512
512
2359296
512
512
1048576
2048
2048
1048576
512
512
2359296
512
512
1048576
2048
2048
2048000
1000


In [10]:
resnet50_model.fc=nn.Sequential(nn.Linear(2048,84),nn.ReLU(),nn.Dropout(0.4),nn.Linear(84,2),nn.LogSoftmax(dim=1))
resnet50_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(resnet50_model.parameters(), lr=0.001)

In [12]:
epochs=10

In [13]:
train_epoch_loss=[]
test_epoch_loss=[]
start_time=time.time()
for e in range(epochs):
    train_batch_loss=[]
    test_batch_loss=[]
    train_acc=0
    test_acc=0
    for batch, (x_train, y_train) in enumerate(train_loader):
        batch+=1
        y_pred_train=resnet50_model(x_train)
#         print(y_pred_train)
#         print(y_pred_train.shape)
        train_acc+=(y_pred_train.argmax(dim=1)==y_train).sum()
        train_loss=criterion(y_pred_train, y_train)
#         print(train_loss)
        if batch%20==0:
            print(f' for epoch {e} and batch  {batch}, training loss = {train_loss} and the accuracy={train_acc*100/(batch*500)}')
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    train_epoch_loss.append(train_acc)
    with torch.no_grad():
        for batch, (x_test, y_test) in enumerate(test_loader):
            batch+=1
            y_pred_test=resnet50_model(x_test)
            test_acc+=(y_pred_test.argmax(dim=1)==y_test).sum()
            test_loss=criterion(y_pred_test, y_test)
            if batch%20==0:
                print(f' for epoch {e} and batch  {batch}, test loss = {test_loss} and the accuracy={test_acc*100/(batch*10)}')
    test_epoch_loss.append(test_acc)
duration=time.time()-start_time

E:\Anaconda\envs\pytorchenv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


 for epoch 0 and batch  20, training loss = 0.15009073913097382 and the accuracy=87.52999877929688
 for epoch 0 and batch  40, training loss = 0.09942071884870529 and the accuracy=91.3550033569336
 for epoch 0 and batch  60, training loss = 0.11880755424499512 and the accuracy=92.89666748046875
 for epoch 0 and batch  20, test loss = 1.4810158014297485 and the accuracy=56.5
 for epoch 0 and batch  40, test loss = 1.9542768001556396 and the accuracy=56.5
 for epoch 0 and batch  60, test loss = 0.9176396131515503 and the accuracy=55.66666793823242
 for epoch 0 and batch  80, test loss = 1.2678539752960205 and the accuracy=54.875
 for epoch 0 and batch  100, test loss = 1.3066078424453735 and the accuracy=55.099998474121094
 for epoch 0 and batch  120, test loss = 1.380629539489746 and the accuracy=55.25
 for epoch 0 and batch  140, test loss = 1.786214828491211 and the accuracy=55.57143020629883
 for epoch 0 and batch  160, test loss = 1.2312467098236084 and the accuracy=55.625
 for epoc

In [14]:
torch.save(resnet50_model.state_dict(),'swami_resnet50.pt')

In [15]:
os.getcwd()

'E:\\to do\\pytorch-for-deep-learning-with-python-bootcamp\\001 PYTORCH_NOTEBOOKS\\PYTORCH_NOTEBOOKS\\03-CNN-Convolutional-Neural-Networks'

In [17]:
train_loss_=[x.item() for x in train_epoch_loss]
test_loss_=[x.item()  for x in test_epoch_loss]
# acc_train_=[x.item()  for x in acc_train]
# acc_test_=[x.item()  for x in acc_test]

In [19]:
losses_custom={'train_losses':train_loss_, 'test_losses':test_loss_,'epochs':epochs,'time':duration}

In [20]:
import json

json = json.dumps(losses_custom)
f = open("Swami_resnet50.json","w")
f.write(json)
f.close()